# Salesforce BLIP3-OCR-200M Data Processing and Reasoning Trace Generation

**Objective:** Utilize the `Salesforce/blip3-ocr-200m` dataset to extract OCR information and then generate reasoning traces using a specified language model. This notebook will be run in a Kaggle SSH environment.

## 1. Setup and Installations

Install necessary Python packages for data handling, Hugging Face interactions, and any other dependencies required by the `multimodal_QRA_pair.py` script and the chosen models.

In [ ]:
# Ensure these packages are installed in your Kaggle environment
# You might need to restart the kernel after running this cell for the first time.
# pip install datasets pandas huggingface_hub pyarrow tqdm
# !git clone https://github.com/minojosh/moremi_reasoning.git
# %cd moremi_reasoning

/kaggle/working/moremi_reasoning


In [1]:
import os
import sys
from pathlib import Path


def set_working_directory():

    # Get the current working directory
    current_dir = Path(os.getcwd())

    # Check if the script is running in a Kaggle environment
    if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
        # Set the working directory to the parent directory of the script / moremi_reasoning
        parent_dir = current_dir.parent
        os.chdir(parent_dir / "working/moremi_reasoning")
        working_dir = sys.path.append(str(parent_dir))
        # print(f"Working directory set to: {working_dir}")
    else:
        # If not in Kaggle, set the working directory to the script's directory
        os.chdir(current_dir)
        sys.path.append(str(current_dir))


set_working_directory()
os.getcwd()

'/kaggle/working/moremi_reasoning'

In [2]:
# login to huggingface_hub with HF_TOKEN from secrets
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
login(HF_TOKEN)

## 2. Import Libraries

In [3]:
import os
import json
import pandas as pd
from datasets import load_dataset
from huggingface_hub import HfApi, snapshot_download
import yaml
from tqdm.auto import tqdm

# Potentially more imports will be needed based on multimodal_QRA_pair.py
# and the specific models used.

## 3. Configuration Loading

Load project configurations. Critical paths like `data_path` and `image_dir` need to be correctly set for the Kaggle environment.

In [4]:
# Path to the configuration file (relative to this notebook)
CONFIG_PATH = "src/config/reasoning_config.yaml"
PROMPTS_PATH = "src/config/reasoning_prompts.yaml"


def load_config(path):
    with open(path, "r") as f:
        return yaml.safe_load(f)


print(f"Loading configuration from: {CONFIG_PATH}")
config = load_config(CONFIG_PATH)
print(f"Loading prompts from: {PROMPTS_PATH}")
prompts = load_config(PROMPTS_PATH)

# --- CRITICAL: REVIEW AND UPDATE THESE PATHS FOR KAGGLE ---
print("\n--- CONFIGURATION REQUIRES YOUR ATTENTION ---")
print(f"Current data_path: {config.get('data_path')}")
print(
    "ACTION: Update 'data_path' in reasoning_config.yaml to the Kaggle path for Salesforce Parquet files."
)

print(f"Current image_dir: {config.get('image_dir')}")
print(
    "ACTION: Update 'image_dir' in reasoning_config.yaml to the Kaggle path for images, if applicable."
)

print(f"Current model_name (for reasoning): {config.get('model_name')}")
print(f"API URL (for reasoning model): {config.get('api_url')}")
print("---------------------------------------------")

# Example of how you might set the Kaggle dataset path if you know it
# KAGGLE_SALESFORCE_OCR_DATASET_PATH = "/kaggle/input/salesforce-blip3-ocr-200m"
# config['data_path'] = os.path.join(KAGGLE_SALESFORCE_OCR_DATASET_PATH, "parquet_files_subfolder_if_any")
# config['image_dir'] = os.path.join(KAGGLE_SALESFORCE_OCR_DATASET_PATH, "image_files_subfolder_if_any")

Loading configuration from: src/config/reasoning_config.yaml
Loading prompts from: src/config/reasoning_prompts.yaml

--- CONFIGURATION REQUIRES YOUR ATTENTION ---
Current data_path: src/data/medpix_report.json
ACTION: Update 'data_path' in reasoning_config.yaml to the Kaggle path for Salesforce Parquet files.
Current image_dir: c:\Users\minom\Pictures\m1\training_images
ACTION: Update 'image_dir' in reasoning_config.yaml to the Kaggle path for images, if applicable.
Current model_name (for reasoning): google/gemini-2.5-pro-preview-03-25
API URL (for reasoning model): https://openrouter.ai/api/v1
---------------------------------------------


## 4. Load Salesforce BLIP3-OCR-200M Dataset

Load the dataset. This might involve using `load_dataset` from Hugging Face `datasets` or directly reading Parquet files if downloaded via `huggingface_hub` or available through Kaggle datasets.

In [11]:
import logging
import sys
import os
from tqdm import tqdm

from huggingface_hub import HfApi, snapshot_download

# Use our previously defined random numbers to select files
prefixes = ["001","047"]  # Original prefixes
# prefixes = [
#     f"{str(n).zfill(3)}" for n in random_numbers
# ]  # Use the random numbers instead
download_list = [f"{prefix}.parquet" for prefix in prefixes]

# Create a directory for data if it doesn't exist
SALESFORCE_DATA_PATH = os.path.join(os.getcwd(), "salesforce_data")
os.makedirs(SALESFORCE_DATA_PATH, exist_ok=True)

# Update the data_path in config for later use
config["data_path"] = SALESFORCE_DATA_PATH
print(f"Updated data_path in config to: {SALESFORCE_DATA_PATH}")


def download_files(download_list, output_dir):
    """
    Function to download files from the Hugging Face Hub.
    Args:
        download_list (list): List of files to download.
        output_dir (str): Directory to save the downloaded files.
    """
    api = HfApi()
    for file in tqdm(download_list, desc="Downloading parquet files"):
        try:
            api.hf_hub_download(
                repo_id="Salesforce/blip3-ocr-200m",
                repo_type="dataset",
                filename=file,
                local_dir=output_dir,
                local_dir_use_symlinks=False,
            )
            print(f"Successfully downloaded {file} to {output_dir}")
        except Exception as e:
            print(f"Error downloading {file}: {e}")


# Download the files
print(f"Downloading {len(download_list)} parquet files: {download_list}")
download_files(download_list, SALESFORCE_DATA_PATH)

Updated data_path in config to: /kaggle/working/moremi_reasoning/salesforce_data


For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


001.parquet:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Successfully downloaded 001.parquet to /kaggle/working/moremi_reasoning/salesforce_data


007.parquet:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

Successfully downloaded 017.parquet to /kaggle/working/moremi_reasoning/salesforce_data


037.parquet:   0%|          | 0.00/4.02G [00:00<?, ?B/s]

Successfully downloaded 037.parquet to /kaggle/working/moremi_reasoning/salesforce_data


047.parquet:   0%|          | 0.00/4.02G [00:00<?, ?B/s]

Successfully downloaded 047.parquet to /kaggle/working/moremi_reasoning/salesforce_data


### 4.1. Specify Parquet File and Load Data

Ensure `SALESFORCE_DATA_PATH` in Cell 7 (Configuration Loading) is correctly set to the directory containing your Parquet files in the Kaggle environment. We'll load one file to start.

In [4]:
!rm -rf salesforce_data/001.parquet
!rm -rf salesforce_data/017.parquet
!rm -rf salesforce_data/047.parquet

In [1]:
import os

os.getcwd()
MOREMI_REASONING_PATH = os.path.join(os.getcwd(), "moremi_reasoning")
os.chdir(MOREMI_REASONING_PATH)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /kaggle/working/moremi_reasoning


In [6]:
# Assuming SALESFORCE_DATA_PATH is a directory containing the .parquet files
# This was defined in Cell 7 based on your config.
import os
import pandas as pd
import sys

os.getcwd()

# Let's list available parquet files if the path is a directory
SALESFORCE_DATA_PATH = os.path.join(os.getcwd(), "salesforce_data")
parquet_file_to_load = None
if (
    SALESFORCE_DATA_PATH
    and os.path.exists(SALESFORCE_DATA_PATH)
    and os.path.isdir(SALESFORCE_DATA_PATH)
):
    all_parquet_files = sorted(
        [
            os.path.join(SALESFORCE_DATA_PATH, f)
            for f in os.listdir(SALESFORCE_DATA_PATH)
            if f.endswith(".parquet")
        ]
    )
    if all_parquet_files:
        parquet_file_to_load = all_parquet_files[0]  # Load the first file
        print(
            f"Found {len(all_parquet_files)} Parquet files. Will load: {parquet_file_to_load}"
        )
    else:
        print(f"ERROR: No .parquet files found in directory: {SALESFORCE_DATA_PATH}")
else:
    print(
        f"ERROR: SALESFORCE_DATA_PATH ('{SALESFORCE_DATA_PATH}') is not a valid directory or does not exist. Please check Cell 7."
    )

# Load the selected Parquet file into a Pandas DataFrame
df = None
if parquet_file_to_load:
    try:
        # Shrink parquet before loading
        %pip install pyarrow -q
        import pyarrow.parquet as pq
        
        # Read the parquet file metadata to see available columns
        parquet_metadata = pq.read_metadata(parquet_file_to_load)
        print(f"Total rows in file: {parquet_metadata.num_rows}")
        
        # Get column names from metadata
        column_names = []
        for i in range(parquet_metadata.num_columns):
            column_names.append(parquet_metadata.schema.names[i])
        print(f"Available columns: {column_names}")
        
        # Read only the first 10 rows and selected columns
        table = pq.read_table(
            parquet_file_to_load, 
            columns=column_names,
            use_threads=True
        )

    except Exception as e:
        print(f"Error loading Parquet file {parquet_file_to_load}: {e}")

Found 2 Parquet files. Will load: /kaggle/working/moremi_reasoning/salesforce_data/007.parquet
Note: you may need to restart the kernel to use updated packages.
Total rows in file: 3896717
Available columns: ['uid', 'face_bboxes', 'url', 'key', 'captions', 'metadata']


In [7]:
# Select 5000 random rows with fixed seed
import numpy as np
import random
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

SEED = 42
N = 7000

# Set seed for reproducibility
np.random.seed(SEED)
random.seed(SEED)

# Get total row count
total_rows = table.num_rows
print(f"Total rows in dataset: {total_rows}")

# Generate random indices
if total_rows > N:
    random_indices = sorted(np.random.choice(total_rows, size=N, replace=False))

    # Use pyarrow directly to read specific rows efficiently
    reader = pq.ParquetFile(parquet_file_to_load)

    # Read in batches and collect only the rows we need
    all_selected_rows = []
    current_row = 0

    for batch in reader.iter_batches(batch_size=10000):
        batch_df = batch.to_pandas()
        batch_size = len(batch_df)

        # Find which random indices fall within this batch
        batch_indices = [
            i - current_row
            for i in random_indices
            if current_row <= i < current_row + batch_size
        ]

        if batch_indices:
            selected_rows = batch_df.iloc[batch_indices]
            all_selected_rows.append(selected_rows)

        current_row += batch_size

        # Early stopping if we've processed all needed indices
        if current_row > max(random_indices):
            break

    # Combine all selected rows
    df = pd.concat(all_selected_rows, ignore_index=True)
else:
    # If we have fewer than N rows, just use the whole table
    df = table.to_pandas()
    print(f"Warning: Requested {N} rows but dataset only has {total_rows} rows.")

# Save to file in chunks to avoid memory issues
output_path = "output.json"
chunk_size = 500
for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i : i + chunk_size]
    mode = "w" if i == 0 else "a"
    chunk.to_json(output_path, orient="records", lines=True, mode=mode)

print(f"Selected {len(df)} random rows from {total_rows} total rows")
print(f"Data saved to {output_path}")
print(df.head())  # Print only the first few rows
# df

Total rows in dataset: 3896717
Selected 7000 random rows from 3896717 total rows
Data saved to output.json
                                uid  \
0  73142d356bceb4a9b16e41f7a78807e9   
1  ea556b0fbaa509f7eadcb690ad672198   
2  97940e449ec693dca50e265b878a97ba   
3  608a2a8216dca37d3bac2650ff131be6   
4  67a9cd91bbcbbd815ff5954536af77e3   

                                         face_bboxes  \
0                                                 []   
1                                                 []   
2                                                 []   
3  [[0.3066563606262207, 0.2672288417816162, 0.55...   
4  [[0.21715471148490906, 0.16724713146686554, 0....   

                                                 url           key  \
0  https://d2840bd96689bd871bd1-41a6b1bb2f8a7c769...  000008120942   
1     https://cv1.litres.ru/pub/c/cover/24166812.jpg  000008129583   
2  https://www.blacktownbuildingsupplies.com.au/w...  000008129783   
3  http://direct.rhapsody.com/imageserver

### 4.2. Explore a Sample Row

Let's examine the content of a single row, focusing on `uid`, `url`, `captions`, and `metadata`.

In [6]:
def explore_sample_row(dataframe, index=0):
    if dataframe is None or dataframe.empty:
        print("DataFrame is not loaded or is empty.")
        return
    if index >= len(dataframe):
        print(
            f"Index {index} is out of bounds for DataFrame of length {len(dataframe)}."
        )
        return

    sample = dataframe.iloc[index]
    print(f"--- Exploring Sample Row (Index: {index}) ---")
    print(f"UID: {sample.get('uid')}")
    print(f"Key: {sample.get('key')}")
    image_url = sample.get("url")
    print(f"Image URL: {image_url}")

    print("\n--- Parsed Captions (from 'captions' field) ---")
    captions_str = sample.get("captions")
    if captions_str:
        try:
            captions_list = json.loads(captions_str)
            for i, cap_obj in enumerate(captions_list):
                print(f"  Caption Entry {i}:")
                print(f"    Granularity: {cap_obj.get('granularity')}")
                print(
                    f"    Include DataComp Raw Cap: {cap_obj.get('include_datacomp_raw_cap')}"
                )
                # Limit printing of very long text fields
                ocr_text = cap_obj.get("text", "")
                print(
                    f"    Text: {ocr_text[:500] + ('...' if len(ocr_text) > 500 else '')}"
                )
        except json.JSONDecodeError as e:
            print(f"  Error parsing 'captions' JSON: {e}")
            print(f"  Raw captions string: {captions_str}")
    else:
        print("  'captions' field is missing or empty.")

    print("\n--- Parsed Metadata (from 'metadata' field) ---")
    metadata_str = sample.get("metadata")
    if metadata_str:
        try:
            metadata_obj = json.loads(metadata_str)
            print(f"  Length (tokens): {metadata_obj.get('length')}")
            print(f"  OCR UID: {metadata_obj.get('uid')}")
            entries = metadata_obj.get("entries", [])
            print(f"  Number of token entries: {len(entries)}")
            if entries:
                print("    First 3 token entries (if available):")
                for i, entry in enumerate(entries[:3]):
                    print(
                        f"      Token {i}: Text='{entry.get('text')}', Confidence={entry.get('confidence')}, BBox={entry.get('bbox')}"
                    )
        except json.JSONDecodeError as e:
            print(f"  Error parsing 'metadata' JSON: {e}")
            print(f"  Raw metadata string: {metadata_str}")
    else:
        print("  'metadata' field is missing or empty.")


if df is not None:
    explore_sample_row(df, 0)  # Explore the first row
    if len(df) > 1:
        explore_sample_row(df, 1)  # Explore the second row if available

--- Exploring Sample Row (Index: 0) ---
UID: 73142d356bceb4a9b16e41f7a78807e9
Key: 000008120942
Image URL: https://d2840bd96689bd871bd1-41a6b1bb2f8a7c769906f5cde19335fe.ssl.cf1.rackcdn.com/thumbnails/3HGGK5H46LM702372/0d58289257f95d4db9562b8e86124ab5.jpg

--- Parsed Captions (from 'captions' field) ---
  Caption Entry 0:
    Granularity: 0
    Include DataComp Raw Cap: False
    Text: The image contains the text "BOCH", the text "20 Year / 200,000 Mile Warranty"
  Caption Entry 1:
    Granularity: 1
    Include DataComp Raw Cap: False
    Text: The image contains the text "BOCH" located at the bottom left corner, the text "20 Year / 200,000 Mile Warranty" located below the center
  Caption Entry 2:
    Granularity: 2
    Include DataComp Raw Cap: False
    Text: The image contains the text "BOCH" located at 18/19 of the image from top to bottom and 2/15 of the image from left to right, the text "20 Year / 200,000 Mile Warranty" located at 18/19 of the image from top to bottom and 10/19

### 4.3. Retrieve Images from `url` key 

Based on the exploration, we'll retrieve or download the images via the url and track the files downloaded by thier `uid` so we can tell which image has a valid image and is accecible on the web.

In [7]:
import requests
from PIL import Image
from io import BytesIO
import os
from tqdm.auto import tqdm
import time

# Create a directory to store the images
IMAGE_DIR = os.path.join(os.getcwd(), "salesforce_images")
os.makedirs(IMAGE_DIR, exist_ok=True)
MAX_IMAGES_TO_DOWNLOAD = 5000  # Limit to prevent excessive downloads


# Update the image_dir in config for later use
config["image_dir"] = IMAGE_DIR
print(f"Images will be saved to: {IMAGE_DIR}")


# Function to download an image from URL
def download_image(url, uid, timeout=10, max_retries=3):
    """
    Downloads an image from a URL and saves it with the uid as filename.

    Args:
        url (str): URL of the image
        uid (str): Unique ID to use as filename
        timeout (int): Timeout in seconds for the request
        max_retries (int): Number of retry attempts

    Returns:
        tuple: (success (bool), path_if_successful (str) or error_message (str))
    """
    if not url:
        return False, "No URL provided"

    image_path = os.path.join(IMAGE_DIR, f"{uid}.jpg")

    # Check if already downloaded
    if os.path.exists(image_path):
        return True, image_path

    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(url, timeout=timeout)
            if response.status_code == 200:
                try:
                    # Verify it's a valid image by opening it with PIL
                    img = Image.open(BytesIO(response.content))
                    img.save(image_path)
                    return True, image_path
                except Exception as e:
                    return False, f"Invalid image data: {str(e)}"
            else:
                retry_count += 1
                if retry_count == max_retries:
                    return False, f"HTTP error: {response.status_code}"
                time.sleep(1)  # Wait before retrying
        except requests.exceptions.Timeout:
            retry_count += 1
            if retry_count == max_retries:
                return False, "Request timed out"
            time.sleep(1)  # Wait before retrying
        except requests.exceptions.RequestException as e:
            return False, f"Request error: {str(e)}"

    return False, "Max retries exceeded"


# Download a sample of images (limit to prevent excessive downloads)
max_images_to_download = MAX_IMAGES_TO_DOWNLOAD

download_results = {}

if df is not None and not df.empty:
    num_to_download = min(len(df), max_images_to_download)
    print(f"Downloading {num_to_download} images...")

    for idx, row in tqdm(
        df.head(num_to_download).iterrows(),
        total=num_to_download,
        desc="Downloading images",
    ):
        uid = row.get("uid")
        url = row.get("url")

        if uid and url:
            success, result = download_image(url, uid)
            download_results[uid] = {"url": url, "success": success, "result": result}

    # Print download statistics
    successful_downloads = sum(1 for res in download_results.values() if res["success"])
    print(
        f"Successfully downloaded {successful_downloads} out of {len(download_results)} images."
    )

    # Print details of a few successful downloads
    successful_items = [item for item in download_results.items() if item[1]["success"]]
    if successful_items:
        print("\nSample of successful downloads:")
        for uid, data in successful_items[:3]:  # Show first 3 successful downloads
            print(f"  UID: {uid}")
            print(f"  Image path: {data['result']}")
            print(f"  URL: {data['url']}")
            print()

    # Print details of failed downloads
    failed_items = [item for item in download_results.items() if not item[1]["success"]]
    if failed_items:
        print("Sample of failed downloads:")
        for uid, data in failed_items[:3]:  # Show first 3 failed downloads
            print(f"  UID: {uid}")
            print(f"  Error: {data['result']}")
            print(f"  URL: {data['url']}")
            print()

Images will be saved to: /kaggle/working/moremi_reasoning/salesforce_images


Successfully downloaded 3667 out of 5000 images.

Sample of successful downloads:
  UID: ea556b0fbaa509f7eadcb690ad672198
  Image path: /kaggle/working/moremi_reasoning/salesforce_images/ea556b0fbaa509f7eadcb690ad672198.jpg
  URL: https://cv1.litres.ru/pub/c/cover/24166812.jpg

  UID: 97940e449ec693dca50e265b878a97ba
  Image path: /kaggle/working/moremi_reasoning/salesforce_images/97940e449ec693dca50e265b878a97ba.jpg
  URL: https://www.blacktownbuildingsupplies.com.au/wp-content/uploads/2019/01/wtex-small-int-corner-3660mm-250x250.jpg

  UID: 608a2a8216dca37d3bac2650ff131be6
  Image path: /kaggle/working/moremi_reasoning/salesforce_images/608a2a8216dca37d3bac2650ff131be6.jpg
  URL: http://direct.rhapsody.com/imageserver/images/Alb.235102483/170x170.jpg

Sample of failed downloads:
  UID: 73142d356bceb4a9b16e41f7a78807e9
  Error: HTTP error: 404
  URL: https://d2840bd96689bd871bd1-41a6b1bb2f8a7c769906f5cde19335fe.ssl.cf1.rackcdn.com/thumbnails/3HGGK5H46LM702372/0d58289257f95d4db9562b8e8

In [8]:
# save zip file of images
!zip -r salesforce_images.zip salesforce_images

updating: salesforce_images/ (stored 0%)
updating: salesforce_images/608a2a8216dca37d3bac2650ff131be6.jpg (deflated 4%)
updating: salesforce_images/67a9cd91bbcbbd815ff5954536af77e3.jpg (deflated 1%)
updating: salesforce_images/97940e449ec693dca50e265b878a97ba.jpg (deflated 14%)
updating: salesforce_images/3692945861512b8cf02cb096ccd893bd.jpg (deflated 16%)
  adding: salesforce_images/fd20cdac1668df9126182cdc1c2aa279.jpg (deflated 3%)
  adding: salesforce_images/5b142cd6cce63636c6223eb972d549b0.jpg (deflated 1%)
  adding: salesforce_images/02765a0cc0b060e2959d66b13a47b07b.jpg (deflated 2%)
  adding: salesforce_images/e17acff24896d6515915c1abe1c1525c.jpg (deflated 7%)
  adding: salesforce_images/275908e0af620b4cacdadbc519d06ade.jpg (deflated 2%)
  adding: salesforce_images/0bd2c1a7e78140a04ae2624365e26916.jpg (deflated 2%)
  adding: salesforce_images/ebe6d74452abae0d29aed3ac1cb4f90f.jpg (deflated 4%)
  adding: salesforce_images/bf7226163eeb3ce391acaf2fe6bcb120.jpg (deflated 0%)
  adding:

### 4.4. Generate OCR QA Pairs for Reasoning Pipeline

Now we'll use the OCR Data Bridge to convert our Salesforce OCR data into the format expected by the multimodal QRA reasoning pipeline. This is the critical integration step that connects OCR data processing with reasoning trace generation.

In [ ]:
# get all file names from the `salesforce_test` directory
import os
import sys

sys.path.append("../src")
from ocr_data_bridge import OCRDataBridge

# Initialize the bridge
bridge = OCRDataBridge()

# # def get_all_file_names(directory):
# #     if not os.path.exists(directory):
# #         print(f"Directory {directory} does not exist.")
# #         return []

# #     return [
# #         f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))
# #     ]


# # Get all file names from the salesforce_images directory
# DIRECTORY = "../src/data/salesforce_test"
# all_image_files = get_all_file_names(DIRECTORY)
# # save the list to a text file
IMAGE_DIR = os.path.join(os.getcwd(), "salesforce_images")

sample_test_images = """
6f87885362ba58d8ebee51378da401a7.jpg
72732f1e5509203199c0757a82c3c321.jpg
646d9028775dc62287bf6d3c5b15d98c.jpg
ab8aa466524a67eededf62f4d07ffff1.jpg
a95178e7a3c21aca2e9f6431be30db13.jpg
4a14eb0a4611bd8f11b71c674045b229.jpg
98e83726f08aaa4420a3c819ea60f3ca.jpg
ea26764e02d3e54cdf965c4e1c385b2a.jpg
4dc59f552f47dd1d8c33e30a250ce53e.jpg
e77334efd7d832af5b0a8237d8cf8b05.jpg
0af25029f56d7856e9c9260755058d31.jpg
"""
# Save the sample test images to a text file
with open("all_image_files.txt", "w") as f:
    f.write(sample_test_images)
# Read the file names from the text file
with open("all_image_files.txt", "r") as f:
    all_image_files = [line.strip() for line in f.readlines() if line.strip()]

# Print the number of files found
files_found = len(all_image_files)
print(f"Found {files_found} files in the directory: {IMAGE_DIR}")
sample_size = files_found


def get_ocr_text_for_qna(
    sample_row, desired_granularity=0, prefer_no_raw_datacomp=True
):
    if sample_row is None:
        return None

    captions_str = sample_row.get("captions")
    if not captions_str:
        return None

    try:
        captions_list = json.loads(captions_str)
        # Try to find the exact granularity, optionally filtering by include_datacomp_raw_cap
        for cap_obj in captions_list:
            if cap_obj.get("granularity") == desired_granularity:
                if (
                    prefer_no_raw_datacomp
                    and cap_obj.get("include_datacomp_raw_cap") == True
                ):
                    continue
                return cap_obj.get("text")

        # Fallback: if exact granularity with preference not found, try without preference
        if prefer_no_raw_datacomp:
            for cap_obj in captions_list:
                if cap_obj.get("granularity") == desired_granularity:
                    return cap_obj.get("text")

        # Fallback: if desired_granularity not found at all, try to get any text from granularity 0 or 5
        for cap_obj in captions_list:
            if cap_obj.get("granularity") == 0:
                if (
                    prefer_no_raw_datacomp
                    and cap_obj.get("include_datacomp_raw_cap") == True
                ):
                    continue
                return cap_obj.get("text")
        for cap_obj in captions_list:
            if cap_obj.get("granularity") == 5:
                if (
                    prefer_no_raw_datacomp
                    and cap_obj.get("include_datacomp_raw_cap") == True
                ):
                    continue
                # Clean the XML-like tags from granularity 5 for a cleaner answer
                text_g5 = cap_obj.get("text")
                if text_g5:
                    return re.sub(
                        r"<ocr>([^<]+)</ocr><bbox>[^<]+</bbox>", r"\1", text_g5
                    ).strip()
                return None  # Should not happen if text_g5 exists

        # Final fallback: return the first caption's text if any
        if captions_list:
            return captions_list[0].get("text")

    except json.JSONDecodeError:
        return None
    return None


if df is not None and not df.empty:
    sample_for_qna = df.iloc[0]

    # --- Configuration for Q&A Generation ---
    # You can change this to experiment with different OCR outputs
    # Granularity 0: Basic text extraction.
    # Granularity 5: Text with OCR tags and bounding boxes (will be cleaned by get_ocr_text_for_qna).
    CHOSEN_GRANULARITY_FOR_ANSWER = 0
    # --- End Configuration ---

    ground_truth_ocr_text = get_ocr_text_for_qna(
        sample_for_qna, CHOSEN_GRANULARITY_FOR_ANSWER
    )

    if ground_truth_ocr_text:
        question = "What is all the text visible in this image?"
        image_uid = sample_for_qna.get("uid")
        image_url_for_qna = sample_for_qna.get("url")

        print(f"--- Generated Q&A for Image UID: {image_uid} ---")
        print(f"Image URL: {image_url_for_qna}")
        print(f"Question: {question}")
        print(
            f"Ground-Truth Answer (from Granularity {CHOSEN_GRANULARITY_FOR_ANSWER}, cleaned):\n{ground_truth_ocr_text}"
        )

        # This is the data structure one item of your preprocessed list would look like for multimodal_QRA_pair.py
        # (assuming multimodal_QRA_pair.py's main data loading is bypassed or adapted)
        prepared_data_item = {
            "process_id": image_uid,  # Or some other unique ID
            "Open-ended Verifiable Question": question,
            "Ground-True Answer": ground_truth_ocr_text,
            "img_urls": [
                image_url_for_qna
            ],  # multimodal_QRA_pair.py expects a list of URLs
        }
        print("\nPrepared data item structure for the reasoning script:")
        print(json.dumps(prepared_data_item, indent=2))
    else:
        print(
            f"Could not extract suitable OCR text for Q&A from sample row (UID: {sample_for_qna.get('uid')})."
        )
else:
    print("DataFrame not loaded or empty, skipping Q&A formulation.")

# Make sure re is imported if not already
import re
import json
import random

# Since we have the file names in all_image_files, let's randomly sample one
random_file = random.choice(all_image_files) if all_image_files else None

if random_file:
    # Extract UID from filename (assuming filename contains the UID)
    uid = random_file.split(".")[0]  # Remove extension

    print(f"Selected random file: {random_file}")
    print(f"Extracted UID: {uid}")

    # Create a mock sample row with minimal required fields
    sample_row = {
        "uid": uid,
        "url": f"{DIRECTORY}/{random_file}",
        "captions": json.dumps(
            [
                {
                    "granularity": 0,
                    "text": f"Sample OCR text for {uid}",
                    "include_datacomp_raw_cap": False,
                },
                {
                    "granularity": 5,
                    "text": f"<ocr>Sample OCR text for {uid}</ocr><bbox>0,0,100,100</bbox>",
                    "include_datacomp_raw_cap": False,
                },
            ]
        ),
    }

    # Use the existing function to get OCR text
    ocr_text = get_ocr_text_for_qna(sample_row, CHOSEN_GRANULARITY_FOR_ANSWER)

    # Create sample Q&A
    question = "What is all the text visible in this image?"
    prepared_data_item = {
        "process_id": uid,
        "Open-ended Verifiable Question": question,
        "Ground-True Answer": ocr_text,
        "img_urls": [sample_row["url"]],
    }

    print("\nGenerated sample Q&A:")
    print(json.dumps(prepared_data_item, indent=2))
else:
    print("No files found to sample.")

# Create QA pairs from our Salesforce data
print("=== Generating OCR QA Pairs ===")
print(f"Working with {len(df)} rows of Salesforce OCR data")
print(f"Image directory: {IMAGE_DIR}")

# Generate QA pairs using different granularities
for granularity in [0, 5]:  # Test both granularities
    print(f"\n--- Processing Granularity {granularity} ---")

    qa_pairs = bridge.create_ocr_qa_pairs(
        salesforce_df=df,
        image_dir=IMAGE_DIR,
        num_samples=50,  # Start with small sample for testing
        granularity=granularity,
        seed=42,
    )

    print(f"Generated {len(qa_pairs)} QA pairs")

    # Validate the QA pairs
    validation = bridge.validate_qa_pairs(qa_pairs)
    print(f"Validation Results:")
    print(f"  Total pairs: {validation['total_pairs']}")
    print(f"  Valid pairs: {validation['valid_pairs']}")
    print(f"  Issues found: {len(validation['issues'])}")

    if validation["issues"]:
        print("\nFirst few issues:")
        for issue in validation["issues"][:3]:
            print(f"  - Pair {issue['pair_index']}: {issue['issues']}")

    # Save for reasoning pipeline
    output_file = f"ocr_qa_pairs_granularity_{granularity}.json"
    bridge.save_for_reasoning_pipeline(qa_pairs, output_file)

    # Show sample QA pair
    if qa_pairs:
        print(f"\nSample QA Pair (Granularity {granularity}):")
        sample = qa_pairs[0]
        print(f"  Question: {sample['Open-ended Verifiable Question']}")
        print(
            f"  Answer: {sample['Ground-True Answer'][:200]}{'...' if len(sample['Ground-True Answer']) > 200 else ''}"
        )
        print(f"  Image URLs: {sample['img_urls']}")
        print(f"  Process ID: {sample['process_id']}")

print("\n=== OCR QA Pair Generation Complete ===")
print("Files generated:")
print("- ocr_qa_pairs_granularity_0.json")
print("- ocr_qa_pairs_granularity_5.json")
print("\nThese files are now ready for the reasoning pipeline!")

Directory ../src/data/salesforce_test does not exist.
Found 0 files in the directory: ../src/data/salesforce_test
--- Generated Q&A for Image UID: 73142d356bceb4a9b16e41f7a78807e9 ---
Image URL: https://d2840bd96689bd871bd1-41a6b1bb2f8a7c769906f5cde19335fe.ssl.cf1.rackcdn.com/thumbnails/3HGGK5H46LM702372/0d58289257f95d4db9562b8e86124ab5.jpg
Question: What is all the text visible in this image?
Ground-Truth Answer (from Granularity 0, cleaned):
The image contains the text "BOCH", the text "20 Year / 200,000 Mile Warranty"

Prepared data item structure for the reasoning script:
{
  "process_id": "73142d356bceb4a9b16e41f7a78807e9",
  "Open-ended Verifiable Question": "What is all the text visible in this image?",
  "Ground-True Answer": "The image contains the text \"BOCH\", the text \"20 Year / 200,000 Mile Warranty\"",
  "img_urls": [
    "https://d2840bd96689bd871bd1-41a6b1bb2f8a7c769906f5cde19335fe.ssl.cf1.rackcdn.com/thumbnails/3HGGK5H46LM702372/0d58289257f95d4db9562b8e86124ab5.jpg"


In [ ]:
# Import the OCR Question Generator
import sys
import os
sys.path.append('../src')
from ocr_question_generator import OCRQuestionGenerator

# Initialize the question generator
ocr_generator = OCRQuestionGenerator()

# Generate diverse OCR questions for testing
print("=== OCR Question Generation for Testing ===")
test_questions = ocr_generator.generate_diverse_questions(15)

print("\nGenerated OCR Test Questions:")
for i, question in enumerate(test_questions, 1):
    difficulty = ocr_generator.assess_difficulty(question)
    category = ocr_generator.categorize_question(question)
    print(f"{i}. [{difficulty.upper()}] [{category}] {question}")

# Create comprehensive test dataset
if df is not None and not df.empty:
    # Convert DataFrame to list of dictionaries for processing
    salesforce_data = df.to_dict('records')
    
    # Create OCR test data with diverse questions
    ocr_test_data = ocr_generator.create_ocr_test_data(salesforce_data, num_samples=10)
    
    print(f"\n=== Created {len(ocr_test_data)} OCR Test Samples ===")
    
    # Display sample test data
    for i, test_item in enumerate(ocr_test_data[:3], 1):
        print(f"\nSample {i}:")
        print(f"  Question: {test_item['question']}")
        print(f"  Difficulty: {test_item['metadata']['difficulty']}")
        print(f"  Type: {test_item['metadata']['question_type']}")
        print(f"  Ground Truth: {test_item['answer'][:200]}..." if len(test_item['answer']) > 200 else f"  Ground Truth: {test_item['answer']}")
    
    # Save test data for multimodal_QRA_pair.py
    test_data_path = "ocr_test_data.json"
    with open(test_data_path, 'w', encoding='utf-8') as f:
        json.dump(ocr_test_data, f, ensure_ascii=False, indent=2)
    
    print(f"\nTest data saved to: {test_data_path}")
    print(f"This file can be used with multimodal_QRA_pair.py for reasoning trace generation")
else:
    print("No DataFrame available for test data creation")

## 5. Running the OCR Reasoning Pipeline

Now that we have properly formatted QA pairs, we can run the multimodal QRA reasoning pipeline with our improved OCR-specific prompts.

In [ ]:
# Placeholder for functionalities from multimodal_QRA_pair.py
# Once multimodal_QRA_pair.py is available, its functions for processing OCR data
# and generating question-reasoning-answer pairs will be called here.

print("ACTION: Provide the 'multimodal_QRA_pair.py' script.")
print("This section cannot be implemented without it.")

# Example structure (highly speculative without the script):
# def process_single_image_ocr(image_path, ocr_data_from_parquet_row):
#     # ... logic from multimodal_QRA_pair.py ...
#     pass

# def generate_reasoning_trace(image_path, question, ocr_text):
#     # ... logic using prompts from prompts.yaml and model from config.yaml ...
#     pass

# for index, row in tqdm(ocr_dataset.iterrows(), total=len(ocr_dataset)):
#     # Assuming ocr_dataset is a pandas DataFrame loaded from Parquet
#     image_id = row['uid'] # or 'key'
#     image_url = row['url']
#     captions_json = row['captions'] # This is a JSON string
#     metadata_json = row['metadata'] # This is a JSON string
    
#     # Parse the JSON strings
#     captions_data = json.loads(captions_json)
#     metadata_data = json.loads(metadata_json)
    
#     # Determine image path (this needs robust handling based on how images are stored/accessed in Kaggle)
#     # current_image_path = os.path.join(config.get('image_dir'), f"{image_id}.jpg") # Example, actual extension might vary
    
#     # Select desired OCR granularity from captions_data
#     # For example, granularity 5 (text with OCR tags and bounding boxes)
#     selected_ocr_text = None
#     for cap in captions_data:
#         if cap['granularity'] == 5 and not cap['include_datacomp_raw_cap']:
#             selected_ocr_text = cap['text']
#             break
            
#     if not selected_ocr_text:
#         # Fallback or skip if desired granularity not found
#         # print(f"Granularity 5 not found for {image_id}")
#         continue
        
#     # TODO: Define a question for the image/OCR data
#     # question = "Describe the main subject and any prominent text in this image."
    
#     # if os.path.exists(current_image_path):
#     #     reasoning_data = generate_reasoning_trace(current_image_path, question, selected_ocr_text)
#     #     # Save or process reasoning_data
#     # else:
#     #     # print(f"Image not found: {current_image_path}")
#     pass # End of loop placeholder

# Update the reasoning config to point to our generated OCR data
import yaml

# Load and update config
with open('../src/config/reasoning_config.yaml', 'r') as f:
    reasoning_config = yaml.safe_load(f)

# Update paths for our OCR data
reasoning_config['data_path'] = 'ocr_qa_pairs_granularity_0.json'  # Use granularity 0 data
reasoning_config['image_dir'] = IMAGE_DIR
reasoning_config['limit_num'] = 10  # Start with small batch for testing
reasoning_config['max_search_attempts'] = 2  # Reasonable for testing
reasoning_config['efficient_search'] = True
reasoning_config['num_process'] = 1  # Single process for debugging

# Save updated config
with open('../src/config/reasoning_config.yaml', 'w') as f:
    yaml.dump(reasoning_config, f, default_flow_style=False)

print("Updated reasoning configuration:")
print(f"  Data path: {reasoning_config['data_path']}")
print(f"  Image directory: {reasoning_config['image_dir']}")
print(f"  Model: {reasoning_config['model_name']}")
print(f"  API URL: {reasoning_config['api_url']}")
print(f"  Limit: {reasoning_config['limit_num']} items")

print("\n=== Ready to Run Reasoning Pipeline ===")
print("To run the reasoning pipeline:")
print("1. Ensure your API_KEY environment variable is set")
print("2. Run: cd ../src && python multimodal_QRA_pair.py")
print("\nThe pipeline will:")
print("- Load OCR QA pairs")
print("- Apply Chain of Thought reasoning with OCR-specific prompts")
print("- Generate reasoning traces for each OCR question")
print("- Save results with verification status")

# Test API key availability
import os
api_key = os.getenv('API_KEY')
if api_key:
    print(f"\n✓ API_KEY is set (length: {len(api_key)})")
else:
    print("\n⚠️ WARNING: API_KEY environment variable not set!")
    print("Set it with: export API_KEY='your_openrouter_api_key'")

# Show what files are expected by the pipeline
expected_files = [
    'ocr_qa_pairs_granularity_0.json',
    '../src/config/reasoning_config.yaml',
    '../src/config/reasoning_prompts.yaml'
]

print("\n=== File Check ===")
for file_path in expected_files:
    if os.path.exists(file_path):
        print(f"✓ {file_path}")
    else:
        print(f"✗ {file_path} - MISSING!")

print("\n=== Integration Summary ===")
print("Your OCR pipeline now properly interfaces with multimodal_QRA_pair.py:")
print("1. ✓ Fixed config loading paths")
print("2. ✓ Added URL/local image handling")
print("3. ✓ Fixed data structure compatibility")
print("4. ✓ Enhanced prompts for OCR tasks")
print("5. ✓ Generated proper QA pairs")
print("\nYou can now run end-to-end OCR reasoning!")